In [4]:
import pandas as pd
import numpy as np
import pathlib
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoModel, AutoTokenizer, BitsAndBytesConfig
import torch
from utils.embeddings import Embedder
from utils.preprocess import create_sentance_booklet, create_faise_index
from utils.utils import search_content, read_booklets, retrieve_booklet_text
import faiss
from llama_cpp import Llama


/opt/miniconda3/envs/zindi_llm/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Read in Data

In [5]:
pwd  = str(pathlib.Path().cwd().parent.resolve())

In [ ]:
df_booklet = read_booklets((pwd + "/data/data/booklets/"))

Since we need to create embeddings of the text, it is important to know how long text is since it can influence the tokenization for some models (can end up truncating text)

In [ ]:
df_booklet['textLength'] = [len(text) for text in df_booklet['text']]
print("Maximum Character lenght is: ", df_booklet['textLength'].max())

There paragraphs are long, so we might need to consider spliting text on sentences to make them shorter.

It seem like the following steps will have to be taken:

- embed booklet
- embed search phrase
- use search phrase embedding to search for relevant text in booklet
- retrive all relevant text from booklet
- format search phrase and into prompt for LLM
- Send promt to LLM and return answer

In [ ]:
df_sentances = create_sentance_booklet(df_booklet=df_booklet)

In [ ]:
df_sentances.shape

In [ ]:
df_sentances['textLength'] = [len(text) for text in df_sentances['sentances']]

In [ ]:
df_sentances.columns = ['originalID', 'book', 'sentances', 'textLength']

# Try simple model
I will first use all-mpnet-base-v2  as the sentance embedder and then I will use Llama as the LLM .

- Download: `wget https://huggingface.co/TheBloke/Llama-2-7B-Chat-GGML/resolve/main/llama-2-7b-chat.ggmlv3.q4_1.bin`
- Then run: pip install llama-cpp-python==0.1.78


In [ ]:
# we can not use gpu
torch.set_default_device("cpu")

1. Embed all sentances

In [ ]:
embedding_model = Embedder("sentence-transformers/all-mpnet-base-v2")

In [ ]:
booklet_embeddings = embedding_model.embed(df_sentances['sentances'].values)

2. Creat faiss index for search.

In [ ]:
fastIndex = create_faise_index(booklet_embeddings)

In [ ]:
# Save the index
faiss.write_index(fastIndex, pwd + "/data/data/resources/index.faiss")
df_sentances.to_csv(pwd + "/data/data/resources/booklet_sentences.csv")


In [ ]:
# Read in index
fastIndex = faiss.read_index( pwd + "/data/data/resources/index.faiss")

3. Search embeddings

In [8]:
text = "When should specimens be collected for Anthrax diagnosis, and how should they be prepared, stored, and transported?"

In [ ]:
df_search_results = search_content(query=text, df_sentances=df_sentances, index=fastIndex, embedder=embedding_model, k=10)

In [ ]:
retrieve_booklet_text(df_booklet=df_booklet, ids=[226])

In [6]:
LLM = Llama(model_path=pwd + "/llama-2-7b-chat.ggmlv3.q4_1.bin")

llama.cpp: loading model from /Users/altasaunders/Alta_projects/zindi_llm/llama-2-7b-chat.ggmlv3.q4_1.bin
llama_model_load_internal: format     = ggjt v3 (latest)
llama_model_load_internal: n_vocab    = 32000
llama_model_load_internal: n_ctx      = 512
llama_model_load_internal: n_embd     = 4096
llama_model_load_internal: n_mult     = 256
llama_model_load_internal: n_head     = 32
llama_model_load_internal: n_head_kv  = 32
llama_model_load_internal: n_layer    = 32
llama_model_load_internal: n_rot      = 128
llama_model_load_internal: n_gqa      = 1
llama_model_load_internal: rnorm_eps  = 5.0e-06
llama_model_load_internal: n_ff       = 11008
llama_model_load_internal: freq_base  = 10000.0
llama_model_load_internal: freq_scale = 1
llama_model_load_internal: ftype      = 3 (mostly Q4_1)
llama_model_load_internal: model size = 7B
llama_model_load_internal: ggml ctx size =    0.08 MB
llama_model_load_internal: mem required  = 4017.35 MB (+  256.00 MB per state)
llama_new_context_with_mode

In [9]:
output = LLM(text)


llama_print_timings:        load time = 50043.86 ms
llama_print_timings:      sample time =   290.04 ms /   128 runs   (    2.27 ms per token,   441.32 tokens per second)
llama_print_timings: prompt eval time = 50043.79 ms /    28 tokens ( 1787.28 ms per token,     0.56 tokens per second)
llama_print_timings:        eval time = 267046.77 ms /   127 runs   ( 2102.73 ms per token,     0.48 tokens per second)
llama_print_timings:       total time = 318103.38 ms


In [10]:
output

{'id': 'cmpl-2964c8fb-400d-4e4e-897c-ed97523d6146',
 'object': 'text_completion',
 'created': 1707052411,
 'model': '/Users/altasaunders/Alta_projects/zindi_llm/llama-2-7b-chat.ggmlv3.q4_1.bin',
 'choices': [{'text': '\nzekere.com 2018; 4(2):e00035.\nAnthrax is a severe disease caused by the Bacillus anthracis bacterium, which can affect humans, animals, and plants. Early diagnosis of anthrax is crucial for effective treatment and prevention of further infections. Here are some guidelines on when to collect specimens for anthrax diagnosis, how to prepare them, store them, and transport them:\nWhen to Collect Specimens for Anthrax Diagnosis:\nSpecimens',
   'index': 0,
   'logprobs': None,
   'finish_reason': 'length'}],
 'usage': {'prompt_tokens': 28, 'completion_tokens': 128, 'total_tokens': 156}}